<a href="https://colab.research.google.com/github/thofes/SJ_distance/blob/main/Prepare_Labeled_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

#init Repository - YOLOv5
%cd /content


!git clone https://github.com/thofes/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install


#init Repository - OpenPose
import os
from os.path import exists, join, basename, splitext

#Change the drive to my mounted gdrive
%cd /content

git_repo_url = 'https://github.com/thofes/openpose'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  print("install new CMake becaue of CUDA10")
  cmake_version = 'cmake-3.20.2-linux-x86_64.tar.gz'
  if not exists(cmake_version):
    !wget -q 'https://cmake.org/files/v3.20/{cmake_version}'
  !tar xfz {cmake_version} --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`


In [ ]:
from numpy.core.fromnumeric import shape
#15042022
#detect.py File anpassen (Github- detect_BB_KP_15042022.py)

#Update
# 28 < x < 75 
# abs < 30
# Lückenhafte Bilder werden besser

#Update zurück 20 < x > 75

#aktuell für pseudelabel und nur cropednormalized without BB

#sollte nun final sein - Test 02.04.2022 16:53 - erfolgreich

#für weitere Durchläufe wo bereits exp vorhanden sind

import os
from os.path import exists, join, basename, splitext
import math
from collections import OrderedDict
import json
import numpy as np
import cv2


#Path wo das File mit den Keypoints + Label gespeichert wird
json_path = "/content.gdrive/MyDrive/Data_Präsentation/xx_"

# dictionary to store mapping, labels, and Keypoints
dataset = {
    "labels":[],
    "keypoints":[]
}

#Path der Bilder - am besten Ordnerstruktur so:
  #Hauptordner eines Sprunges 
      #Ordner mit Lables (
      #-FP 
          #Alle Bilder die dem zugeordnet sind
      #-LP 
          #Alle Bilder die dem zugeordnet sind
      #-LF
          #Alle Bilder die dem zugeordnet sind


PATH = "/content.gdrive/MyDrive/Data_Präsentation"
label= []
try:
    label = next(os.walk(PATH))[1]
except StopIteration:
    pass
#data["labels"].append(label)
print(label)

savecounter = 0

for l in label:
  Path_jumpers = PATH + '/' + l
  print(Path_jumpers)
  Jumpers = []
  try:
    Jumpers = next(os.walk(Path_jumpers))[1]
  except StopIteration:
    pass
  print("Anzahl: " , len(Jumpers))
  

  for j in Jumpers:
    Path_Pic = Path_jumpers + '/' + j
    print(Path_Pic)
    
#if exp vorhanden or not
      
    Path_check = Path_Pic + '/exp'


    if os.path.isdir(Path_check):

        print("Done")
    else:

        %cd /content/yolov5
        !python detect.py --weights "/content/yolov5/weights_skijumper_only.pt" --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt




        
        folder_dir = Path_Pic + '/exp/crops/skijumper'
        %cd /content/openpose/build/examples/tutorial_api_python
        !python3 04_keypoints_from_images.py --image_dir '{folder_dir}' --keypoint_scale 3 --no_display true --write_images '{folder_dir}'  #keypoint_scale 0für nicht normalisiert

      
        # im Ordner crops/skijumper/keypoints müssen alle json Files der Reihen anch eingelesen und in das Dictionary mit Label eingefügt werden
        # Einlesen wieviele keypointsxx.json Files im Ordner sind und dann eine For Loop über die gesamte Anzahl
        #with open(Path_Pic + '/exp2/crops/skijumper/keypoints' + xx + '.json', "r") as fp:
            #keypoints_json = json.load(fp)
        #dataset["keypoints"].append(keypoints_json)
        #dataset["labels"].append(label)



        """
        if l == 'FP':
            dataset["labels"].append(0)
        if l == 'LF':
            dataset["labels"].append(1) 
        if l == 'LF':
            dataset["labels"].append(9)
        """
        
        savecounter +=1

        if savecounter%1 == 0:
            #json_path_1 = json_path + 'with_BB_' + str(savecounter-99) + '_' + str(savecounter) + '.json'
            #with open(json_path_1, "w") as fp:
                #json.dump(dataset, fp, indent=4)

            json_path_2 = Path_Pic + '/' + str(savecounter) + '.json'
            with open(json_path_2, "w") as fp:
                json.dump(dataset2, fp, indent=4)

            # dictionary to store mapping, labels, and Keypoints leeren
            dataset = {
                "labels":[],
                "keypoints_with_BB":[]
            }


json_path_2 = json_path + 'last_' + str(savecounter) + '.json'
with open(json_path_2, "w") as fp:
  json.dump(dataset2, fp, indent=4)  
